In [1]:
import cv2
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from selenium import webdriver
import time
from tqdm import tqdm
from selenium.common.exceptions import NoSuchElementException
import warnings

warnings.filterwarnings('ignore')

In [20]:
def new_box(box):
    l_d = [box[:,0].min(), box[:,1].min()]
    l_u = [box[:,0].min(), box[:,1].max()]
    r_d = [box[:,0].max(), box[:,1].min()]
    r_u = [box[:,0].max(), box[:,1].max()]
    return np.array([l_d, l_u, r_u, r_d])
def make_sample(img_name, new_name):
    hsv_min = np.array((33, 33, 33), np.uint8)
    hsv_max = np.array((230, 230, 230), np.uint8)
    img = cv2.imread(img_name)
    hsv = cv2.cvtColor( img, cv2.COLOR_BGR2HSV )
    thresh = cv2.inRange( hsv, hsv_min, hsv_max )
    contours0, hierarchy = cv2.findContours( thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    result = pd.DataFrame()
    for cnt in contours0:
        rect = cv2.minAreaRect(cnt)
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        n_box = new_box(box)
        area = int(rect[1][0]*rect[1][1])
        
        if area > 200 and area < 300000:
            cv2.drawContours(img,[n_box],0,(0,0,255),1)
            result = result.append(pd.DataFrame({
                'name':[new_name],
                'left_d x':[n_box[0,0]],
                'left_d y':[n_box[0,1]],
                'right_u x':[n_box[2,0]],
                'right_u y':[n_box[2,1]],
                'is house':['False']
            }))

    return img, result
def all_img(image_names):
    result_log = pd.DataFrame()
    for i in range(image_names.shape[0]):
        img_name = image_names.iloc[i]['img name']
        img, r = make_sample(img_name, 'IMG'+str(i)+'.png')
        cv2.imwrite('IMG'+str(i)+'.jpg', img)
        result_log = result_log.append(r)
    result_log.set_index('name').to_csv('log.csv')
def get_img_names(f, t):
    result = pd.DataFrame()
    for i in range(f, t + 1):
        result = result.append(pd.DataFrame({
            'img name':[str(i)+'.png']
        }))
    return result
def schreenshot_maker(driver, points):
    for i in range(points.shape[0]):
        point = points.iloc[i]
        lat, lon = point['lat'], point['lon']
        url = 'https://pkk.rosreestr.ru/#/search/'+str(lat)+','+str(lon)+'/17/@e8ia1091'
        driver.get(url)
        time.sleep(3)
        driver.save_screenshot (str(68+i)+".png" )
def poit_maker(lat_begin, lon_begin, n_i, n_j, step_lat, step_lon):
    result = {'lat':[],'lon':[]}
    for i in range(n_i):
        for j in range(n_j):
            result['lat'].append(lat_begin + i*step_lat)
            result['lon'].append(lon_begin + j*step_lon)
    return pd.DataFrame(result)

In [21]:
points = poit_maker(55.686404155093676, 32.252510023800376, 5, 5, 0.005, 0.015)

In [4]:
driver_path = 'C:/Users/Иван/Downloads/chromedriver_win32_new/chromedriver.exe'
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(driver_path, chrome_options=None)

In [22]:
schreenshot_maker(driver, points)

In [23]:
names = get_img_names(0, 92)
all_img(names)